In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage

##Filters
A Filter is a function from an asset and a moment in time to a boolean:
```
F(asset, timestamp) -> boolean
```
In Pipeline, [Filters](https://www.quantopian.com/help#quantopian_pipeline_filters_Filter) are used for narrowing down the set of securities included in a computation or in the final output of a pipeline. There are two common ways to create a `Filter`: comparison operators and `Factor`/`Classifier` methods.

###Comparison Operators
Comparison operators on `Factors` and `Classifiers` produce Filters. Since we haven't looked at `Classifiers` yet, let's stick to examples using `Factors`. The following example produces a filter that returns `True` whenever the latest close price is above $20.

In [2]:
last_close_price = USEquityPricing.close.latest
close_price_filter = last_close_price > 20

And this example produces a filter that returns True whenever the 10-day mean is below the 30-day mean.

In [3]:
mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)
mean_crossover_filter = mean_close_10 < mean_close_30

Remember, each security will get its own `True` or `False` value each day.

###Factor/Classifier Methods
Various methods of the `Factor` and `Classifier` classes return `Filters`. Again, since we haven't yet looked at `Classifiers`, let's stick to `Factor` methods for now (we'll look at `Classifier` methods later). The `Factor.top(n)` method produces a `Filter` that returns `True` for the top `n` securities of a given `Factor`. The following example produces a filter that returns `True` for exactly 200 securities every day, indicating that those securities were in the top 200 by last close price across all known securities.

In [4]:
last_close_price = USEquityPricing.close.latest
top_close_price_filter = last_close_price.top(200)

For a full list of `Factor` methods that return `Filters`, see [this link](https://www.quantopian.com/help#quantopian_pipeline_factors_Factor).

For a full list of `Classifier` methods that return `Filters`, see [this link](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier).

##Dollar Volume Filter
As a starting example, let's create a filter that returns `True` if a security's 30-day average dollar volume is above $10,000,000. To do this, we'll first need to create an `AverageDollarVolume` factor to compute the 30-day average dollar volume. Let's include the built-in `AverageDollarVolume` factor in our imports:

In [5]:
from quantopian.pipeline.factors import AverageDollarVolume

And then, let's instantiate our average dollar volume factor.

In [6]:
dollar_volume = AverageDollarVolume(window_length=30)

By default, `AverageDollarVolume` uses `USEquityPricing.close` and `USEquityPricing.volume` as its `inputs`, so we don't specify them.

Now that we have a dollar volume factor, we can create a filter with a boolean expression. The following line creates a filter returning `True` for securities with a `dollar_volume` greater than 10,000,000:

In [7]:
high_dollar_volume = (dollar_volume > 10000000)

To see what this filter looks like, let's can add it as a column to the pipeline we defined in the previous lesson.

In [8]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30
    
    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = (dollar_volume > 10000000)

    return Pipeline(
        columns={
            'percent_difference': percent_difference,
            'high_dollar_volume': high_dollar_volume
        }
    )

If we make and run our pipeline, we now have a column `high_dollar_volume` with a boolean value corresponding to the result of the expression for each security.

In [9]:
result = run_pipeline(make_pipeline(), '2018-05-05', '2018-05-05')
result

high_dollar_volume  \
2018-05-07 00:00:00+00:00 Equity(2 [HWM])                       True   
                          Equity(21 [AAME])                    False   
                          Equity(24 [AAPL])                     True   
                          Equity(25 [HWM_PR])                  False   
                          Equity(31 [ABAX])                    False   
                          Equity(41 [ARCB])                    False   
                          Equity(52 [ABM])                      True   
                          Equity(53 [ABMD])                     True   
                          Equity(62 [ABT])                      True   
                          Equity(64 [GOLD])                     True   
                          Equity(66 [AB])                      False   
                          Equity(67 [ADSK])                     True   
                          Equity(70 [VBF])                     False   
                          Equity(76 [TAP])                      True   
                          Equity(84 [ACET])                    False   
                          Equity(100 [IEP])                    False   
                          Equity(106 [ACU])                    False   
                          Equity(110 [RAMP])                    True   
                          Equity(112 [ACY])                    False   
                          Equity(114 [ADBE])                    True   
                          Equity(117 [AEY])                    False   
                          Equity(122 [ADI])                     True   
                          Equity(128 [ADM])                     True   
                          Equity(149 [ADX])                    False   
                          Equity(153 [AE])                     False   
                          Equity(154 [AEM])                     True   
                          Equity(157 [AEG])                     True   
                          Equity(161 [AEP])                     True   
                          Equity(166 [AES])                     True   
                          Equity(168 [AET])                     True   
...                                                              ...   
                          Equity(51954 [SDI])                  False   
                          Equity(51955 [CDAY])                  True   
                          Equity(51957 [LASR])                 False   
                          Equity(51959 [ENBA])                 False   
                          Equity(51960 [DOV_WI])               False   
                          Equity(51961 [SMAR])                 False   
                          Equity(51962 [GSHD])                 False   
                          Equity(51963 [ELOX])                 False   
                          Equity(51964 [SJIU])                 False   
                          Equity(51966 [APY])                  False   
                          Equity(51967 [EDNT])                 False   
                          Equity(51968 [FCBP])                 False   
                          Equity(51969 [ECCX])                 False   
                          Equity(51970 [BH_A])                 False   
                          Equity(51971 [PRT])                  False   
                          Equity(51973 [UBX])                  False   
                          Equity(51974 [ASLN])                 False   
                          Equity(51975 [INSP])                 False   
                          Equity(51976 [STXB])                 False   
                          Equity(51977 [ETP_PRC])              False   
                          Equity(51979 [RCP])                  False   
                          Equity(51980 [BCML])                 False   
                          Equity(51981 [JRSH])                 False   
                          Equity(51982 [CBLK])                 False   
                  

##Applying a Screen
By default, a pipeline produces computed values each day for every asset in the Quantopian database. Very often however, we only care about a subset of securities that meet specific criteria (for example, we might only care about securities that have enough daily trading volume to fill our orders quickly). We can tell our Pipeline to ignore securities for which a filter produces `False` by passing that filter to our Pipeline via the `screen` keyword.

To screen our pipeline output for securities with a 30-day average dollar volume greater than $10,000,000, we can simply pass our `high_dollar_volume` filter as the `screen` argument. This is what our `make_pipeline` function now looks like:

In [10]:
def make_pipeline():

    mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10)
    mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30)

    percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

    dollar_volume = AverageDollarVolume(window_length=30)
    high_dollar_volume = dollar_volume > 10000000

    return Pipeline(
        columns={
            'percent_difference': percent_difference
        },
        screen=high_dollar_volume
    )

When we run this, the pipeline output only includes securities that pass the `high_dollar_volume` filter on a given day. For example, running this pipeline on May 5th, 2018 results in an output for ~2,100 securities

In [11]:
result = run_pipeline(make_pipeline(), '2018-05-05', '2018-05-05')
print 'Number of securities that passed the filter: %d' % len(result)
result

Number of securities that passed the filter: 2131


percent_difference
2018-05-07 00:00:00+00:00 Equity(2 [HWM])                  -0.098612
                          Equity(24 [AAPL])                -0.008472
                          Equity(52 [ABM])                 -0.030249
                          Equity(53 [ABMD])                 0.035502
                          Equity(62 [ABT])                 -0.004917
                          Equity(64 [GOLD])                 0.033192
                          Equity(67 [ADSK])                -0.012542
                          Equity(76 [TAP])                 -0.047784
                          Equity(110 [RAMP])                0.015106
                          Equity(114 [ADBE])                0.001488
                          Equity(122 [ADI])                -0.019998
                          Equity(128 [ADM])                 0.013590
                          Equity(154 [AEM])                -0.003354
                          Equity(157 [AEG])                 0.026316
                          Equity(161 [AEP])                 0.011888
                          Equity(166 [AES])                 0.048724
                          Equity(168 [AET])                 0.020595
                          Equity(185 [AFL])                 0.016725
                          Equity(197 [AGCO])               -0.029819
                          Equity(216 [HES])                 0.059710
                          Equity(239 [AIG])                 0.011225
                          Equity(266 [AJG])                 0.011754
                          Equity(270 [AKRX])               -0.215365
                          Equity(300 [ALK])                 0.033350
                          Equity(301 [ALKS])               -0.049247
                          Equity(311 [ALOG])               -0.050908
                          Equity(337 [AMAT])               -0.056599
                          Equity(351 [AMD])                 0.047062
                          Equity(353 [AME])                -0.037491
                          Equity(357 [TWX])                -0.007388
...                                                              ...
                          Equity(51231 [ROKU])             -0.000144
                          Equity(51268 [SWCH])             -0.017044
                          Equity(51281 [CARG])             -0.064894
                          Equity(51306 [QD])               -0.074203
                          Equity(51314 [MDB])              -0.073180
                          Equity(51350 [FSCT])             -0.008387
                          Equity(51455 [SEND])              0.016609
                          Equity(51468 [SAIL])              0.080547
                          Equity(51489 [CNNE])              0.054488
                          Equity(51494 [DLPH])             -0.007710
                          Equity(51576 [CASA])             -0.129790
                          Equity(51578 [LFIN])                   NaN
                          Equity(51618 [NTR])              -0.006542
                          Equity(51634 [AVYA])              0.007491
                          Equity(51649 [ADT])               0.015872
                          Equity(51689 [PAGS])             -0.063878
                          Equity(51734 [VICI])             -0.007753
                          Equity(51806 [CRON])             -0.014971
                          Equity(51815 [GLIB_A])           -0.063835
                          Equity(51832 [ZS])                0.011115
                          Equity(51854 [DBX])              -0.017147
                          Equity(51875 [BILI])             -0.031794
                          Equity(51887 [IQ])                0.007459
                          Equity(51895 [SPOT])              0.036191
                          Equity(51917 [ZUO])              -0.004698
                          Equity(51928 [NVT])              -0.006314
                          Equity(51934 [EAF])   

##Inverting a Filter
The `~` operator is used to invert a filter, swapping all `True` values with `Falses` and vice-versa. For example, we can write the following to filter for low dollar volume securities:

In [12]:
low_dollar_volume = ~high_dollar_volume

This will return `True` for all securities with an average dollar volume below or equal to $10,000,000 over the last 30 days.

In the next lesson, we will look at combining filters.